In [1]:
#load libraries
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 10)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

In [35]:
#Add columns
pulsar_data <- read_csv("pulsar_data.csv", 
                        col_names = c("mean_integrated_profile", 
                                      "stand_dev_integrated_profile", 
                                      "exc_kurtosis_integrated_profile", 
                                      "skew_integrated_profile",
                                      "mean_dmsnr", 
                                      "stand_dev_dmsnr", 
                                      "exc_kurtosis_dmsnr", 
                                      "skew_dmsnr", "class")) 

pulsar_data <- pulsar_data |>
    mutate(class = as_factor(class))
#pulsar_data 

pulsar_train <- pulsar_data |>
    select(class, skew_dmsnr, skew_integrated_profile) 
#pulsar_train

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 5) |>
  set_engine("kknn") |>
  set_mode("classification")
#knn_spec

knn_fit <- knn_spec |>
  fit(class ~ ., data = pulsar_train)
#knn_fit

unscaled_pulsar <- pulsar_data |>
    select(class, skew_dmsnr, skew_integrated_profile) 
#unscaled_pulsar

up_recipe <- recipe(class ~ ., data = unscaled_pulsar) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors()) |>
    prep()
#up_recipe

scaled_pulsar <- bake(up_recipe, unscaled_pulsar)
#scaled_pulsar

#Class proportions in pulsar dataset
nrow(scaled_pulsar)

#Imbalanced dataset
pulsar_proportions <- pulsar_data |>
    group_by(class) |>
    summarize(n = n())
pulsar_proportions


Rows: 17898 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (9): mean_integrated_profile, stand_dev_integrated_profile, exc_kurtosis...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] 17898

class,n
<fct>,<int>
0,16259
1,1639
